# T-SQL Procedures 101: From Keywords to Complex Procedures

This comprehensive notebook teaches T-SQL procedures from the ground up, starting with individual keywords and clauses, then building to complex stored procedures.

## Learning Approach
- **Keyword Purpose**: What each T-SQL keyword does
- **Clause Breakdown**: Line-by-line explanations
- **Block Purpose**: Overall goal of each code section
- **Implementation Details**: How and why it works
- **Progressive Complexity**: From simple statements to full procedures

---

## Setup: Create Practice Database and Tables

First, let's create tables that we'll use throughout our procedure examples.

In [ ]:
-- Create database for procedure practice
CREATE DATABASE ProcedureDB;
USE ProcedureDB;

-- Create Employees table for examples
CREATE TABLE Employees (
    EmployeeID INT IDENTITY(1,1) PRIMARY KEY,
    FirstName NVARCHAR(50) NOT NULL,
    LastName NVARCHAR(50) NOT NULL,
    Department NVARCHAR(50),
    Salary DECIMAL(10,2),
    HireDate DATE DEFAULT GETDATE(),
    IsActive BIT DEFAULT 1
);

-- Insert sample data
INSERT INTO Employees (FirstName, LastName, Department, Salary, HireDate)
VALUES 
    ('John', 'Smith', 'IT', 75000, '2020-01-15'),
    ('Jane', 'Doe', 'HR', 65000, '2021-03-10'),
    ('Mike', 'Johnson', 'Finance', 70000, '2019-08-22');

## 1. DECLARE Keyword - Variable Declaration

### Keyword Purpose
**DECLARE** creates variables to store temporary data during script execution.

### Syntax Breakdown
```sql
DECLARE @VariableName DataType [= InitialValue];
```

### Implementation Details
- Variables must start with `@` symbol
- Must specify data type (INT, NVARCHAR, DECIMAL, etc.)
- Can optionally set initial value
- Variables are local to the batch/procedure where declared

In [ ]:
-- Basic DECLARE examples with different data types

-- Integer variable
DECLARE @EmployeeCount INT;

-- String variable with initial value
DECLARE @DepartmentName NVARCHAR(50) = 'IT';

-- Decimal variable for salary calculations
DECLARE @AverageSalary DECIMAL(10,2);

-- Date variable
DECLARE @CurrentDate DATE = GETDATE();

-- Boolean variable
DECLARE @IsProcessed BIT = 0;

-- Display declared variables
SELECT 
    @EmployeeCount AS EmployeeCount,
    @DepartmentName AS DepartmentName,
    @AverageSalary AS AverageSalary,
    @CurrentDate AS CurrentDate,
    @IsProcessed AS IsProcessed;

## 2. SET Keyword - Variable Assignment

### Keyword Purpose
**SET** assigns values to variables after they've been declared.

### Syntax Breakdown
```sql
SET @VariableName = Value;
```

### Implementation Details
- Can assign literal values, expressions, or query results
- SET assigns one value at a time
- If query returns multiple rows, SET takes the last value
- Use SELECT for multiple variable assignments

In [ ]:
-- SET examples with different value types

DECLARE @Message NVARCHAR(100);
DECLARE @TotalEmployees INT;
DECLARE @MaxSalary DECIMAL(10,2);

-- Assign literal string
SET @Message = 'Processing employee data...';

-- Assign result from COUNT function
SET @TotalEmployees = (SELECT COUNT(*) FROM Employees);

-- Assign result from MAX function
SET @MaxSalary = (SELECT MAX(Salary) FROM Employees);

-- Display results
SELECT 
    @Message AS ProcessMessage,
    @TotalEmployees AS TotalEmployees,
    @MaxSalary AS MaxSalary;

PRINT 'Variables assigned successfully!';

## 3. SELECT INTO Variables - Multiple Assignment

### Keyword Purpose
**SELECT INTO** assigns multiple variables in a single statement from query results.

### Syntax Breakdown
```sql
SELECT @Var1 = Column1, @Var2 = Column2 FROM Table WHERE Condition;
```

### Implementation Details
- More efficient than multiple SET statements
- Can assign multiple variables from one row
- If no rows match, variables remain unchanged
- If multiple rows match, uses last row values

In [ ]:
-- SELECT INTO examples for multiple variable assignment

DECLARE @EmpFirstName NVARCHAR(50);
DECLARE @EmpLastName NVARCHAR(50);
DECLARE @EmpSalary DECIMAL(10,2);
DECLARE @EmpDepartment NVARCHAR(50);

-- Assign multiple variables from one employee record
SELECT 
    @EmpFirstName = FirstName,
    @EmpLastName = LastName,
    @EmpSalary = Salary,
    @EmpDepartment = Department
FROM Employees
WHERE EmployeeID = 1;

-- Display the assigned values
SELECT 
    @EmpFirstName AS FirstName,
    @EmpLastName AS LastName,
    @EmpSalary AS Salary,
    @EmpDepartment AS Department;

-- Calculate and assign aggregate values
DECLARE @AvgSalary DECIMAL(10,2);
DECLARE @MinSalary DECIMAL(10,2);
DECLARE @EmpCount INT;

SELECT 
    @AvgSalary = AVG(Salary),
    @MinSalary = MIN(Salary),
    @EmpCount = COUNT(*)
FROM Employees
WHERE Department = 'IT';

SELECT @AvgSalary AS AvgSalary, @MinSalary AS MinSalary, @EmpCount AS EmployeeCount;

## 4. IF-ELSE Conditional Logic

### Keyword Purpose
**IF-ELSE** executes different code blocks based on conditions.

### Syntax Breakdown
```sql
IF condition
BEGIN
    -- Code when condition is true
END
ELSE
BEGIN
    -- Code when condition is false
END
```

### Implementation Details
- Condition must evaluate to TRUE/FALSE
- BEGIN-END blocks group multiple statements
- ELSE is optional
- Can nest IF-ELSE statements

In [ ]:
-- IF-ELSE examples with different conditions

DECLARE @EmployeeCount INT;
DECLARE @Message NVARCHAR(100);

-- Get employee count
SELECT @EmployeeCount = COUNT(*) FROM Employees;

-- Simple IF-ELSE based on count
IF @EmployeeCount > 5
BEGIN
    SET @Message = 'Large company with ' + CAST(@EmployeeCount AS NVARCHAR(10)) + ' employees';
    PRINT @Message;
END
ELSE
BEGIN
    SET @Message = 'Small company with ' + CAST(@EmployeeCount AS NVARCHAR(10)) + ' employees';
    PRINT @Message;
END

-- Nested IF-ELSE for salary analysis
DECLARE @HighSalaryCount INT;
SELECT @HighSalaryCount = COUNT(*) FROM Employees WHERE Salary > 70000;

IF @HighSalaryCount > 0
BEGIN
    PRINT 'Company has high-paid employees';
    
    IF @HighSalaryCount > 2
    BEGIN
        PRINT 'Multiple high earners detected';
    END
    ELSE
    BEGIN
        PRINT 'Few high earners';
    END
END
ELSE
BEGIN
    PRINT 'No high-paid employees found';
END

## 5. WHILE Loop - Repetitive Operations

### Keyword Purpose
**WHILE** repeats code blocks as long as a condition remains true.

### Syntax Breakdown
```sql
WHILE condition
BEGIN
    -- Code to repeat
    -- Update condition variable to avoid infinite loop
END
```

### Implementation Details
- Condition checked before each iteration
- Must modify condition variable inside loop
- Use BREAK to exit loop early
- Use CONTINUE to skip to next iteration

In [ ]:
-- WHILE loop examples

-- Simple counter loop
DECLARE @Counter INT = 1;
DECLARE @Result NVARCHAR(200) = '';

WHILE @Counter <= 5
BEGIN
    SET @Result = @Result + 'Step ' + CAST(@Counter AS NVARCHAR(2)) + ' ';
    SET @Counter = @Counter + 1;
END

PRINT 'Loop result: ' + @Result;

-- Process employees one by one using WHILE
DECLARE @CurrentEmpID INT = 1;
DECLARE @MaxEmpID INT;
DECLARE @EmpName NVARCHAR(100);

SELECT @MaxEmpID = MAX(EmployeeID) FROM Employees;

PRINT 'Processing employees:';

WHILE @CurrentEmpID <= @MaxEmpID
BEGIN
    -- Check if employee exists (IDs might not be consecutive)
    IF EXISTS (SELECT 1 FROM Employees WHERE EmployeeID = @CurrentEmpID)
    BEGIN
        SELECT @EmpName = FirstName + ' ' + LastName 
        FROM Employees 
        WHERE EmployeeID = @CurrentEmpID;
        
        PRINT 'Processing: ' + @EmpName;
    END
    
    SET @CurrentEmpID = @CurrentEmpID + 1;
END

PRINT 'All employees processed!';

## 6. Simple Stored Procedure - Combining Keywords

### Procedure Purpose
**CREATE PROCEDURE** packages T-SQL code into reusable, named blocks.

### Syntax Breakdown
```sql
CREATE PROCEDURE ProcedureName
AS
BEGIN
    -- Procedure body using keywords we learned
END
```

### Implementation Details
- Procedures are stored in database
- Can be executed multiple times
- SET NOCOUNT ON prevents row count messages
- Use BEGIN-END to group statements

In [ ]:
-- Create simple procedure using keywords we learned

CREATE PROCEDURE GetEmployeeStats
AS
BEGIN
    -- SET NOCOUNT ON prevents extra result sets
    SET NOCOUNT ON;
    
    -- DECLARE variables for calculations
    DECLARE @TotalEmployees INT;
    DECLARE @AvgSalary DECIMAL(10,2);
    DECLARE @HighEarners INT;
    DECLARE @StatusMessage NVARCHAR(200);
    
    -- SELECT INTO to assign multiple variables
    SELECT 
        @TotalEmployees = COUNT(*),
        @AvgSalary = AVG(Salary)
    FROM Employees;
    
    -- SET to assign high earners count
    SET @HighEarners = (SELECT COUNT(*) FROM Employees WHERE Salary > @AvgSalary);
    
    -- IF-ELSE to create status message
    IF @HighEarners > (@TotalEmployees / 2)
    BEGIN
        SET @StatusMessage = 'Company has many high earners';
    END
    ELSE
    BEGIN
        SET @StatusMessage = 'Company has balanced salary distribution';
    END
    
    -- Return results
    SELECT 
        @TotalEmployees AS TotalEmployees,
        @AvgSalary AS AverageSalary,
        @HighEarners AS HighEarners,
        @StatusMessage AS StatusMessage;
        
    PRINT 'Employee statistics calculated successfully!';
END;

-- Execute the procedure
EXEC GetEmployeeStats;

## 7. Procedure with Parameters - Input and Output

### Parameter Purpose
**Parameters** make procedures flexible by accepting input values and returning output values.

### Syntax Breakdown
```sql
CREATE PROCEDURE ProcName
    @InputParam DataType,
    @OutputParam DataType OUTPUT
AS
BEGIN
    -- Use @InputParam in logic
    -- SET @OutputParam = value
END
```

### Implementation Details
- Input parameters receive values when procedure is called
- OUTPUT parameters return values to caller
- Parameters can have default values
- Must specify OUTPUT keyword for output parameters

In [ ]:
-- Create procedure with input and output parameters

CREATE PROCEDURE AnalyzeDepartment
    @DepartmentName NVARCHAR(50),           -- Input parameter
    @MinSalary DECIMAL(10,2) = 0,           -- Input with default value
    @EmployeeCount INT OUTPUT,              -- Output parameter
    @AverageSalary DECIMAL(10,2) OUTPUT,    -- Output parameter
    @StatusCode INT OUTPUT                  -- Output parameter
AS
BEGIN
    SET NOCOUNT ON;
    
    -- DECLARE local variables
    DECLARE @DeptExists BIT = 0;
    DECLARE @TotalSalary DECIMAL(12,2);
    
    -- Check if department exists
    IF EXISTS (SELECT 1 FROM Employees WHERE Department = @DepartmentName)
    BEGIN
        SET @DeptExists = 1;
    END
    
    -- IF-ELSE logic based on department existence
    IF @DeptExists = 1
    BEGIN
        -- SELECT INTO to get department statistics
        SELECT 
            @EmployeeCount = COUNT(*),
            @TotalSalary = SUM(Salary)
        FROM Employees 
        WHERE Department = @DepartmentName 
          AND Salary >= @MinSalary;
        
        -- Calculate average (avoid division by zero)
        IF @EmployeeCount > 0
        BEGIN
            SET @AverageSalary = @TotalSalary / @EmployeeCount;
            SET @StatusCode = 1; -- Success
            
            PRINT 'Department analysis completed for: ' + @DepartmentName;
        END
        ELSE
        BEGIN
            SET @AverageSalary = 0;
            SET @StatusCode = 2; -- No employees meet criteria
            
            PRINT 'No employees found meeting salary criteria';
        END
    END
    ELSE
    BEGIN
        -- Department doesn't exist
        SET @EmployeeCount = 0;
        SET @AverageSalary = 0;
        SET @StatusCode = 0; -- Department not found
        
        PRINT 'Department not found: ' + @DepartmentName;
    END
END;

-- Execute procedure with parameters
DECLARE @EmpCount INT;
DECLARE @AvgSal DECIMAL(10,2);
DECLARE @Status INT;

EXEC AnalyzeDepartment 
    @DepartmentName = 'IT',
    @MinSalary = 50000,
    @EmployeeCount = @EmpCount OUTPUT,
    @AverageSalary = @AvgSal OUTPUT,
    @StatusCode = @Status OUTPUT;

-- Display results
SELECT 
    @EmpCount AS EmployeeCount,
    @AvgSal AS AverageSalary,
    @Status AS StatusCode,
    CASE @Status
        WHEN 0 THEN 'Department not found'
        WHEN 1 THEN 'Analysis successful'
        WHEN 2 THEN 'No employees meet criteria'
    END AS StatusDescription;